In [1]:
from transformers import AutoImageProcessor, AutoModel
import torch
from torch.utils.data import DataLoader
from src.model import SiameseDino
from src.training import evaluate
from src.data import CachedCollection, LazyLoadCollection, extractPaths, train_test_split, make_transform
from pathlib import Path

/home/timothee/Documents/programmation/DIHT/.env/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/timothee/Documents/programmation/DIHT/.env/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment

In [2]:
base_model_name = "facebook/dinov3-vitb16-pretrain-lvd1689m"
processor = AutoImageProcessor.from_pretrained(base_model_name)
dinov3_model = AutoModel.from_pretrained(
    base_model_name,
    dtype=torch.float32
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
hidden_dim = 512
output_dim = 128
device = torch.device("cuda" if torch.backends.mps.is_available() else "cpu")
siamese_model = SiameseDino(dinov3_model, hidden_dim, output_dim)
siamese_model.load_state_dict(torch.load("../model_checkpoints/smooth-cloud-114.pth"))
_ = siamese_model.to(device)

In [4]:
root_path = Path("../data/original_data")
images_paths, labels = extractPaths(root_path)
train_paths, test_paths, train_labels, test_labels = train_test_split(images_paths, labels, 0.3, random_state=42)
train_dataset = CachedCollection(train_paths, train_labels, make_transform())
test_dataset = CachedCollection(test_paths, test_labels, make_transform())
train_dataloader = DataLoader(train_dataset)
test_dataloader = DataLoader(test_dataset)

In [5]:
print(evaluate(siamese_model, processor, train_dataloader, test_dataloader, [1, 3, 5]))

{'recall@1': 0.5588235294117647, 'recall@3': 0.6764705882352942, 'recall@5': 0.7647058823529411}


In [7]:
print(evaluate(siamese_model, processor, train_dataloader, test_dataloader, [1, 3, 5]))

{'recall@1': 0.5490196078431373, 'recall@3': 0.7058823529411765, 'recall@5': 0.7647058823529411}


In [10]:
print(evaluate(siamese_model, processor, train_dataloader, test_dataloader, [1, 3, 10]))

{'recall@1': 0.5, 'recall@3': 0.6470588235294118, 'recall@10': 0.9019607843137255}
